In [1]:
%%bash
chia version
cdv --version
python --version

1.6.1b4.dev7
cdv, version 1.1.2
Python 3.10.6


# Clawback Scenario
## block height 5

In [2]:
%%bash
chia show -s

Network: simulator0    Port: 58444   RPC Port: 18422
Node ID: 38401aad891c6ecd90a91150989b51e9c86d844e07387ed77338beda85b199e7
Genesis Challenge: eb8c4d20b322be8d9fddbf9412016bdffe9a2901d7edb0e364e94266d0e095f7
Current Blockchain Status: Full Node Synced

Peak: Hash: c9f0f719fef8a1766f0446a89d6a68e1e662a9c1fb088797711ba61d740d632c
      Time: Mon Sep 26 2022 17:12:16 +07                  Height:          4

Estimated network space: 29.284 MiB
Current difficulty: 1024
Current VDF sub_slot_iters: 1024

  Height: |   Hash:
        4 | c9f0f719fef8a1766f0446a89d6a68e1e662a9c1fb088797711ba61d740d632c
        3 | 802d397f4c91d8ab74b9ad7d470b14b2db0eb93b8fc2f1733cb34d2765f1bce2
        2 | 1eef946c9154e000587facb15732114d1d029c4b294553bd1cb25db36ef28163
        1 | 742bd7a0408ef45eb697f9aa84d5a5145e789924ce24690867acbddf79a5bc69


In [3]:
# chia libraries
from blspy import (PrivateKey, AugSchemeMPL, G1Element, G2Element)
from chia.types.blockchain_format.program import Program
from chia.wallet.puzzles import (p2_delegated_puzzle_or_hidden_puzzle)
# utils & tic tac toe helper code
import sys
sys.path.insert(0, "../code")
from utils import (load_program, print_program, print_puzzle, print_json)
import singleton_utils
import tic_tac_toe

# puzzles
tic_tac_toe_puzzle = load_program("../code/tic-tac-toe.clsp", ["../code"])
coin_puzzle = load_program("../code/coin.clsp", ["../code"])
terminate_puzzle = load_program("../code/terminate-game.clsp", ["../code"])

waiting_room_puzzle = load_program("../code/waiting-room.clsp", ["../code"])
play_amount = 1_000_000_000_000

In [4]:
# from setup-sim notebook
# alice 4070016934
# bob 861183802
alice_sk = PrivateKey.from_bytes(bytes.fromhex("67959bf0ade73bc2aa69cab59d0aa1fded890696f178393f7740b330a808f7c6"))
alice_pk = G1Element.from_bytes(bytes.fromhex("96f75bf45d09491e8a5adaedbdb458f6ac46d9b6507a788fe58349f6eb2a56bfbc72bdd70a09eceb060b552f10fb1010"))
alice_puzzle_hash = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(alice_pk).get_tree_hash()
bob_pk = G1Element.from_bytes(bytes.fromhex("b1bc8cb3442693d6093df74dc975f81b74ac1438290341a13feb94af6f08fd613fcd509abb773612b5fe68851b56f64a"))
bob_puzzle_hash = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(bob_pk).get_tree_hash()

# prepare players info
player_one_info = Program.to([alice_pk, alice_puzzle_hash])
player_two_info = Program.to([bob_pk, bob_puzzle_hash])

In [5]:
from chia.util.bech32m import encode_puzzle_hash
## alice
alice_waiting_room_puzzle = waiting_room_puzzle.curry(
    alice_puzzle_hash, 
    alice_pk,
    alice_pk, 
    bob_pk
)
alice_waiting_room_puzzle_hash = alice_waiting_room_puzzle.get_tree_hash()
alice_waiting_room_address = encode_puzzle_hash(alice_waiting_room_puzzle_hash, "txch")
alice_waiting_room_address

'txch1e3pp9eefvar8tje7h5mt2hy0epacyuyynyy53w9adtu56p7nhjxq737a5a'

# Create Alice's Waiting Room Coin

In [6]:
%%bash
chia wallet send -f 4070016934 -t txch1e3pp9eefvar8tje7h5mt2hy0epacyuyynyy53w9adtu56p7nhjxq737a5a -a 1.000000000001

Submitting transaction...
Transaction submitted to nodes: [{'peer_id': '38401aad891c6ecd90a91150989b51e9c86d844e07387ed77338beda85b199e7', 'inclusion_status': 'SUCCESS', 'error_msg': None}]
Run 'chia wallet get_transaction -f 4070016934 -tx 0x7d335df62d41ae1098d865fc1729ecc4bf8dc432599255fd675a5bf14b40a5d4' to get status


In [7]:
%%bash
cdv decode txch1e3pp9eefvar8tje7h5mt2hy0epacyuyynyy53w9adtu56p7nhjxq737a5a

cc4212e729674675cb3ebd36b55c8fc87b827084990948b8bd6af94d07d3bc8c


In [8]:
%%bash
cdv sim -n tic-tac-toe farm
cdv rpc coinrecords --by puzzlehash "cc4212e729674675cb3ebd36b55c8fc87b827084990948b8bd6af94d07d3bc8c"

Farmed 1 Transaction blocks
Block Height is now: 6
[
    {
        "coin": {
            "amount": 1000000000001,
            "parent_coin_info": "0xdf2cf850264c22b8bfe7da999b56b8a6689c9ce25be52d0807fe12ed8c9e1b5b",
            "puzzle_hash": "0xcc4212e729674675cb3ebd36b55c8fc87b827084990948b8bd6af94d07d3bc8c"
        },
        "coinbase": false,
        "confirmed_block_index": 6,
        "spent_block_index": 0,
        "timestamp": 1665157285
    }
]


## Clawback

In [9]:
from chia.rpc.full_node_rpc_client import FullNodeRpcClient
from chia.util.config import load_config
from chia.util.default_root import DEFAULT_ROOT_PATH
from chia.util.ints import uint16

config = load_config(DEFAULT_ROOT_PATH, "config.yaml")
selected_network = config["selected_network"]
genesis_challenge = config["network_overrides"]["constants"][selected_network]["GENESIS_CHALLENGE"]

self_hostname = config["self_hostname"] # localhost
full_node_rpc_port = config["full_node"]["rpc_port"] # 8555
genesis_challenge

'eb8c4d20b322be8d9fddbf9412016bdffe9a2901d7edb0e364e94266d0e095f7'

In [10]:
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
coin_records = await full_node_client.get_coin_records_by_puzzle_hash(alice_waiting_room_puzzle_hash)
full_node_client.close()
await full_node_client.await_closed()

alice_waiting_room_puzzle_coin = coin_records[0].coin
alice_waiting_room_puzzle_coin

Coin { parent_coin_info: df2cf850264c22b8bfe7da999b56b8a6689c9ce25be52d0807fe12ed8c9e1b5b, puzzle_hash: cc4212e729674675cb3ebd36b55c8fc87b827084990948b8bd6af94d07d3bc8c, amount: 1000000000001 }

In [11]:
from utils import print_json
from chia.types.coin_spend import CoinSpend

#  (defconstant CLAWBACK -1)
solution = Program.to([
    0,0,0
])

coin_spend = CoinSpend(
        alice_waiting_room_puzzle_coin,
        alice_waiting_room_puzzle,
        solution
)
print_json(coin_spend.to_json_dict())

{
    "coin": {
        "amount": 1000000000001,
        "parent_coin_info": "0xdf2cf850264c22b8bfe7da999b56b8a6689c9ce25be52d0807fe12ed8c9e1b5b",
        "puzzle_hash": "0xcc4212e729674675cb3ebd36b55c8fc87b827084990948b8bd6af94d07d3bc8c"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff82017fffff01ff04ffff04ff14ffff04ffff02ff16ffff04ff02ffff04ff82017fffff04ff5fffff04ff81bfffff04ff17ffff04ff2fff8080808080808080ff808080ffff02ffff03ffff09ff0bff1780ffff01ff04ffff04ff12ffff04ff1affff01ff8601d1a94a2001808080ff8080ff8080ff018080ffff01ff04ffff04ff1cffff01ff648080ffff04ffff04ff12ffff04ff05ffff01ff8600e8d4a51000808080ffff04ffff04ff08ffff04ff0bffff04ff05ff80808080ff8080808080ff0180ffff04ffff01ffff32ff3d52ffff33a0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9ffff0bff05ffff02ff1effff04ff02ffff04ffff04ff0bffff04ff17ffff04ffff04ffff01ff8467616d658b7469632d7461632d746f65ffff04ffff04ffff018570315f706bff2f80ffff04ffff04ffff018570325f706bff5f80ff80808080ff80808080ff808080

In [15]:
from chia.types.spend_bundle import SpendBundle


message: bytes = alice_puzzle_hash
sig: G2Element = AugSchemeMPL.sign(
    alice_sk,
    message
    + alice_waiting_room_puzzle_coin.name()
    #+ bytes.fromhex(genesis_challenge),
    + bytes.fromhex("ccd5bb71183532bff220ba46c268991a3ff07eb358e8255a65c30a2dce0e5fbb")
)

spend_bundle = SpendBundle([coin_spend], sig)
print_json(spend_bundle.to_json_dict(include_legacy_keys = False, exclude_modern_keys = False))

{
    "aggregated_signature": "0xa319e81319dfc823f174c6164fa22a5e79c9e4b093128881a2d8d834bdf67aa94eee3480d483e9894da874b654e949d108a465a2120563ccc68066cdbf2f0290c9fd9c91cbf94e59813174b1001ea245695931d77288afb330ec46755fd697ea",
    "coin_spends": [
        {
            "coin": {
                "amount": 1000000000001,
                "parent_coin_info": "0xdf2cf850264c22b8bfe7da999b56b8a6689c9ce25be52d0807fe12ed8c9e1b5b",
                "puzzle_hash": "0xcc4212e729674675cb3ebd36b55c8fc87b827084990948b8bd6af94d07d3bc8c"
            },
            "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff82017fffff01ff04ffff04ff14ffff04ffff02ff16ffff04ff02ffff04ff82017fffff04ff5fffff04ff81bfffff04ff17ffff04ff2fff8080808080808080ff808080ffff02ffff03ffff09ff0bff1780ffff01ff04ffff04ff12ffff04ff1affff01ff8601d1a94a2001808080ff8080ff8080ff018080ffff01ff04ffff04ff1cffff01ff648080ffff04ffff04ff12ffff04ff05ffff01ff8600e8d4a51000808080ffff04ffff04ff08ffff04ff0bffff04ff05ff80808080ff8080808080ff0180f

## Clawback is not available yet

In [16]:
%%bash
cdv sim -n tic-tac-toe farm --blocks 10
chia rpc full_node get_blockchain_state | jq ".blockchain_state.peak.height"

Farmed 10 Transaction blocks
Block Height is now: 26
26


In [17]:
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
result = await full_node_client.push_tx(spend_bundle)
full_node_client.close()
await full_node_client.await_closed()
print(result)

{'status': 'PENDING', 'success': True}


## Clawback is available

In [18]:
%%bash
cdv sim -n tic-tac-toe start -wr # restart to clear pending tx
cdv sim -n tic-tac-toe farm --blocks 10
chia rpc full_node get_blockchain_state | jq ".blockchain_state.peak.height"

chia_full_node_simulator: stopped
chia_full_node_simulator: started
chia_wallet: stopped
chia_wallet: started
Farmed 10 Transaction blocks
Block Height is now: 36
36


In [19]:
# check if the waiting room coin is spent
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
coin_records = await full_node_client.get_coin_records_by_puzzle_hash(alice_waiting_room_puzzle_hash)
full_node_client.close()
await full_node_client.await_closed()

assert coin_records[0].spent_block_index == 0

In [20]:
%%bash
cdv sim -n tic-tac-toe farm --blocks 90
chia rpc full_node get_blockchain_state | jq ".blockchain_state.peak.height"

Farmed 90 Transaction blocks
Block Height is now: 126
126


In [21]:
# push tx again when the block height is greater than 100
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
result = await full_node_client.push_tx(spend_bundle)
full_node_client.close()
await full_node_client.await_closed()
print(result)

{'status': 'SUCCESS', 'success': True}


In [22]:
%%bash
cdv sim -n tic-tac-toe farm --blocks 10
chia rpc full_node get_blockchain_state | jq ".blockchain_state.peak.height"

Farmed 10 Transaction blocks
Block Height is now: 136
136


In [23]:
# check if the waiting room coin is spent at the 100+ block
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
coin_records = await full_node_client.get_coin_records_by_puzzle_hash(
    alice_waiting_room_puzzle_hash
)
full_node_client.close()
await full_node_client.await_closed()

assert coin_records[0].spent_block_index > (100 + coin_records[0].confirmed_block_index)
waiting_room_coin_id = coin_records[0].coin.name()
coin_records[0].to_json_dict()

{'coin': {'parent_coin_info': '0xdf2cf850264c22b8bfe7da999b56b8a6689c9ce25be52d0807fe12ed8c9e1b5b',
  'puzzle_hash': '0xcc4212e729674675cb3ebd36b55c8fc87b827084990948b8bd6af94d07d3bc8c',
  'amount': 1000000000001},
 'confirmed_block_index': 6,
 'spent_block_index': 127,
 'coinbase': False,
 'timestamp': 1665157285}

In [24]:
# check if the return coin with 1 XCH is unspent & created,by the waiting room coin
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
coin_records = await full_node_client.get_coin_records_by_puzzle_hash(
    puzzle_hash = alice_puzzle_hash,
    include_spent_coins = False
)
full_node_client.close()
await full_node_client.await_closed()

return_coin_record = next((cr for cr in coin_records if cr.coin.parent_coin_info == waiting_room_coin_id), None)
assert return_coin_record is not None
return_coin_record.to_json_dict()

{'coin': {'parent_coin_info': '0x3b2f59f7a55e5ea62eee8d04f4a5d08497bffd756a33cc0feab0659fd3d39713',
  'puzzle_hash': '0xd210b94e1c190321d74e68fa697a8da3dce5846bbaf9f2783ba345c431123387',
  'amount': 1000000000000},
 'confirmed_block_index': 127,
 'spent_block_index': 0,
 'coinbase': False,
 'timestamp': 1665157587}